##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Word embeddings

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/word_embeddings">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/word_embeddings.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: Cộng đồng TensorFlow Việt Nam đã dịch các tài liệu này từ nguyên bản tiếng Anh.
Vì bản dịch này dựa trên sự cố gắng từ các tình nguyện viên, nên không thể đám bảo luôn bám sát
[Tài liệu chính thức bằng tiếng Anh](https://www.tensorflow.org/?hl=en).
Nếu bạn có đề xuất để cải thiện bản dịch này, vui lòng tạo PR đến repository trên GitHub của [tensorflow/docs](https://github.com/tensorflow/docs)

Để đăng ký dịch hoặc duyệt lại nội dung bản dịch, các bạn hãy liên hệ 
[docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

Chủ đề của hướng dẫn lần này sẽ xoay quanh word embedding. Hướng dẫn sẽ bao gồm code để huấn luyện word embedding hoàn toàn từ đầu trên một tập dữ liệu nhỏ và trực quan hóa các embedding thu được với [Embedding Projector](http://projector.tensorflow.org) (như hình bên dưới).

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=1" alt="Screenshot of the embedding projector" width="400"/>
 
## Biểu diễn văn bản dưới dạng số

Các mô hình máy học nhận đầu vào là các vector (dãy các số). Khi làm việc với văn bản, điều đầu tiên chúng ta cần thực hiện là tìm cách chuyển đổi các chuỗi kí tự sang số (hay "vector hóa" văn bản) trước khi đưa vào mô hình. Trong phần này, chúng ta sẽ đi qua ba cách để thực hiện mục tiêu trên.

### One-hot encoding

Cách làm đầu tiên chúng ta có thể nghĩ tới là encode "one-hot" từng từ trong danh sách từ vựng. Xét câu "The cat sat on the mat". Danh sách từ vựng (hay các từ phân biệt) trong câu này là (cat, mat, on, sat, the). Để biểu diễn mỗi từ, ta tạo một vector 0 có độ dài bằng với số lượng từ trong danh sách từ vựng và thay 1 vào vị trí tương ứng của từ cần biểu diễn. Cách làm này được minh họa bằng sơ đồ bên dưới.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/one-hot.png?raw=1" alt="Diagram of one-hot encodings" width="400" />

Để  thu được vector encode của một câu, ta chỉ việc nối các vector one-hot của từng từ trong câu.

Lưu ý: Cách làm này không hiệu quả vì các vector thu được từ phương pháp one-hot encoding đều là các vector thưa (nghĩa là hầu hết các phần tử đều là 0). Hãy tưởng tượng nếu chúng ta có 10,000 từ trong danh sách từ vựng. Để thực hiện one-hot encoding một từ, ta cần tạo ra một vector mà ở đó 99.9% các phần tử đều là 0.

### Encode mỗi từ bằng một số phân biệt

Cách làm thứ hai mà ta có thể thử là sử dụng các số phân biệt để encode từng từ. Xét ví dụ trên, ta có thể chọn số 1 để encode cho từ "cat", số 2 cho từ "mat" và cứ như thế. Lúc này, chúng ta có thể encode cả câu "The cat sat on the mat" bằng một vector dày đặc, chẳng hạn [5, 1, 4, 3, 5, 2]. Cách làm này mang lại hiệu quả hơn vì ta thu được một vector dày đặc (tất cả các phần tử đều khác 0) thay vì vector thưa.

Tuy nhiên, cách encode này có hai điểm yếu:

* Các số dùng để encode là hoàn toàn ngẫu nhiên (không thể hiện được mối quan hệ giữa các từ với nhau).

* Encode bằng số gây khó khăn cho mô hình. Ví dụ trong thuật toán phân loại tuyến tính, với mỗi thuộc tính mô hình cần học một trọng số tương ứng. Tuy nhiên vì hai từ tương đồng nhau chưa chắc cách encode của chúng sẽ tương tự nhau, trọng số mà mô hình học được từ sự tương đồng trong cách biểu diễn các từ sẽ không còn ý nghĩa.

### Word embedding

Word embedding cung cấp cho ta cách thức biểu diễn từ hiệu quả và dày đặc mà ở đó các từ tương đồng sẽ được encode tương tự nhau. Hơn nữa, chúng ta sẽ không phải trực tiếp xác định cách encode. Mỗi embedding là một vector số thực dày đặc (độ dài của vector sẽ do người dùng xác định). Thay vì trực tiếp xác định các giá trị cho embedding, các giá trị này hoàn toàn có thể huấn luyện được (bằng cách cho mô hình học các trọng số trong quá trình huấn luyện, tương tự như cách học các trọng số đối với lớp dense). Word embedding thường được sử dụng có số chiều là 8 (đối với các tập dữ liệu nhỏ) và có thể lên đến 1024 khi làm việc với các tập dữ liệu lớn. Embedding có số chiều càng lớn thì càng có khả năng biểu diễn được các mối quan hệ "sâu" giữa các từ, tuy nhiên cũng đòi hỏi một lượng lớn dữ liệu để học. 

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding2.png?raw=1" alt="Diagram of an embedding" width="400"/>

Trên đây là sơ đồ của một word embedding. Mỗi từ được biểu diễn bằng một vector số thực 4 chiều. Ta cũng có thể hình dung word embedding như một "bảng tra". Sau khi các trọng số đã được học xong, ta có thể encode từng từ bằng cách tra vào bảng để thu được vector dày đặc là biểu diễn của từ tương ứng.

## Chuẩn bị

In [ ]:
!pip install tf-nightly
import tensorflow as tf

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## Sử dụng lớp Embedding
Thư viện Keras giúp việc sử dụng word embedding trở nên vô cùng đơn giản. Sau đây, chúng ta sẽ cùng tìm hiểu về [lớp Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) được hỗ trợ trong thư viện này.

Lớp Embedding có thể được hiểu như một bảng tra cho phép ánh xạ từ một chỉ mục (đại diện cho một từ nhất định) sang một vector dày đặc (embedding của từ đó). Số chiều (hay độ dài) của embedding là tham số mà bạn có thể tùy biến để phù hợp với bài toán của mình, tương tự như việc tùy biển số lượng neuron khác nhau trong một lớp Dense.

In [ ]:
embedding_layer = layers.Embedding(1000, 5)

Khi khởi tạo một lớp Embedding, các trọng số hỗ trợ việc embedding sẽ được gán các giá trị ngẫu nhiên (tương tự như quá trình khởi tạo ở các lớp khác). Các trọng số này sẽ dần được điều chỉnh thông qua quá trình truyền ngược ở giai đoạn huấn luyện mô hình. Sau khi hoàn tất huấn luyện, các word embedding có thể encode sự tương đồng giữa các từ một cách tương đối (kết quả của việc huấn luyện mô hình trên một bài toán cụ thể).

Nếu truyền một số nguyên vào lớp embedding này, ta sẽ thu được một vector tương ứng thuộc bảng embedding:

In [ ]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

Đối với các bài toán xử lí văn bản hay chuỗi văn bản liên tiếp (sequence), lớp Embedding nhận đầu vào là một tensor số nguyên 2 chiều có kích thước `(số lượng mẫu, độ dài chuỗi)`. Các chuỗi trong dữ liệu đầu vào có thể có độ dài bất kì. Ví dụ, ta có thể đưa vào lớp embedding trên một batch có kích thước `(32, 10)` (gồm 32 chuỗi có độ dài 10) hoặc `(64, 15)` (gồm 64 chuỗi có độ dài 15).

Tensor trả về sẽ có nhiều hơn một chiều so với dữ liệu đầu vào. Tập hợp các vector embedding được xếp vào chiều cuối cùng. Chẳng hạn, nếu ta đưa vào một batch có kích thước `(2, 3)` thì sẽ nhận được kết quả với kích thước `(2, 3, N)`.

In [ ]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

<!-- When given a batch of sequences as input, an embedding layer returns a 3D floating point tensor, of shape `(samples, sequence_length, embedding_dimensionality)`. To convert from this sequence of variable length to a fixed representation there are a variety of standard approaches. You could use an RNN, Attention, or pooling layer before passing it to a Dense layer. This tutorial uses pooling because it's simplest. The [Text Classification with an RNN](text_classification_rnn.ipynb) tutorial is a good next step. -->

Khi nhận đầu vào là một batch các chuỗi, lớp embedding sẽ trả về một tensor số thực 3 chiều có kích thước `(số lượng mẫu, độ dài chuỗi, số chiều embedding)`. Có nhiều cách hỗ trợ chuyển đổi chuỗi có độ dài bất kì trên sang một biểu diễn có kích thước cố định để đưa vào lớp dense: sử dụng RNN, Attention hoặc lớp pooling. Hướng dẫn này sử dụng lớp pooling vì đây được xem phương pháp đơn giản nhất. Ngoài ra, bạn có thể tìm hiểu thêm ở [Hướng dẫn phân loại văn bản với RNN](text_classification_rnn.ipynb).

## Học embedding từ đầu

Trong hướng dẫn này, chúng ta sẽ tiến hành huấn luyện mô hình phân loại sắc thái dựa trên các bình luận phim ở trang IMDB. Mô hình sẽ học cách embedding hoàn toàn từ đầu. Chúng ta sẽ phải thực hiện tiền xử lí trên bộ dữ liệu được cho.

Xem thêm về cách tải dữ liệu văn bản tại [Hướng dẫn tải dữ liệu văn bản](../load_data/text.ipynb).

In [ ]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

Để lấy danh sách từ vựng, ta truy cập vào encoder của tập dữ liệu (`tfds.features.text.SubwordTextEncoder`).

Kí hiệu "\_" được dùng để biểu diễn khoảng trắng. Danh sách từ vựng sẽ bao gồm cả các từ toàn vẹn (kết thúc bằng "\_") lẫn các từ không toàn vẹn (có thể ghép với nhau tạo thành từ có nghĩa): 

In [ ]:
encoder = info.features['text'].encoder
encoder.subwords[:20]

<!-- Movie reviews can be different lengths. We will use the `padded_batch` method to standardize the lengths of the reviews. -->

Bình luận phim có thể có độ dài khác nhau. Do đó, chúng ta sẽ sử dụng `padded_batch` để chuẩn hóa độ dài của các bình luận.

In [ ]:
train_data

In [ ]:
train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes=([None],[]))
test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes=([None],[]))

Lưu ý: Phiên bản **TensorFlow 2.2** không yêu cầu khai báo tham số padded_shapes. Ở chế độ mặc định, dữ liệu đầu vào sẽ được đệm thêm các phần tử sao cho tất cả dữ liệu ở mọi chiều đều có độ dài bằng nhau và bằng với độ dài lớn nhất theo chiều được xét của batch đó.

In [ ]:
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

Như mong đợi, các bình luận dưới dạng văn bản đã được encode thành các số nguyên (mỗi số nguyên đại diện cho một từ toàn vẹn hoặc không toàn vẹn trong danh sách từ vựng).

Các số 0 được thêm vào ở cuối là kết quả của việc chuẩn hóa độ dài mặc định đã được đề cập ở trên.

In [ ]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()

### Xây dựng mô hình đơn giản

Để định nghĩa kiến trúc mô hình, ta sử dụng [Keras Sequential API](../../guide/keras). Trong hướng dẫn này này, ta sẽ xây dựng một mô hình đơn giản theo kiểu "Continous bag of words":

* Lớp Embedding nhận danh sách từ vựng đã được encode bằng số và thực hiện tìm kiếm vector embedding đối với mỗi từ. Các vector này được học thông qua quá trình huấn luyện. Tập hợp các vector embedding thu tạo thành một chiều mới trong dữ liệu đầu ra. Kết quả thu được sẽ có kích thước `(kích thước batch, độ dài chuỗi, số chiều embedding)`.

* Với mỗi mẫu trong batch, lớp GlobalAveragePooling1D sẽ trả về một vector có kích thước cố thước cố định là kết quả của việc tính trung bình dọc theo độ dài chuỗi. Đây được xem là cách làm đơn giản nhất để  xử lý dữ liệu đầu vào có kích thước không cố định.

* Các vector (lúc này đều đã tương đồng nhau về mặt kích thước) tiếp tục được đưa vào một lớp Dense gồm 16 unit ẩn.

* Lớp cuối cùng sẽ chứa các kết nối hoàn toàn tới một node đầu ra duy nhất. Hàm kích hoạt được sử dụng là hàm sigmoid. Kết quả trả về sẽ là một số thực nằm trong khoảng từ 0 đến 1. Số thực này đại diện cho xác suất (hay độ tin cậy) một bình luận mang ý nghĩa tích cực.

Chú ý: Mô hình trên không sử dụng kỹ thuật mask. Do đó, các phần tử 0 thêm vào trong quá trình chuẩn hóa độ dài cũng được xem như một phần của dữ liệu đầu vào. Như vậy, kết quả đầu ra có thể bị ảnh huởng bởi độ dài của phần đệm. Để khắc phục, xem thêm [Hướng dẫn mask và đệm dữ liệu](../../guide/keras/masking_and_padding).

In [ ]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

model.summary()

### Biên dịch và huấn luyện mô hình

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

Với cách làm đã nêu, mô hình đạt độ chính xác khoảng 88% trên tập kiểm thử (để ý rằng mô hình của ta đang bị overfit vì độ chính xác thu được trên tập huấn luyện cao hơn rất nhiều).

In [ ]:
import matplotlib.pyplot as plt

history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss=history_dict['loss']
val_loss=history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()

## Lấy các embedding đã được học

Tiếp theo, ta sẽ lấy các word embedding đã được học trong quá trình huấn luyện. Tập hợp các embedding này là một ma trận có kích thước `(số từ trong danh sách từ vựng, số chiều embedding)`.

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

Thực hiện lưu lại các trọng số này. Biết rằng để trực quan hóa các embedding thu được với [Embedding Projector](http://projector.tensorflow.org), ta cần hai tệp dữ liệu: một tệp chứa các vector embedding và một tệp chứa meta data (là các từ trong danh sách từ vựng).

In [ ]:
import io

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

Nếu đang chạy hướng dẫn này trên [Colaboratory](https://colab.research.google.com), bạn có thể sử dụng đoạn chương trình sau để tải các tệp này về máy (hoặc sử dụng trình duyệt tệp bằng cách chọn *View -> Table of contents -> File browser*).

In [ ]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

## Trực quan hóa các embedding

Để thực hiện trực quan hóa các embedding thu được, trước tiên ta cần tải chúng lên chương trình embedding projector.

Mở [Embedding Projector](http://projector.tensorflow.org/) (hoặc chạy TensorBoard ở local).

* Chọn "Load data".

* Tải lên hai tệp mà ta đã tạo ở trên gồm: `vecs.tsv` và `meta.tsv`.

<!-- The embeddings you have trained will now be displayed. You can search for words to find their closest neighbors. For example, try searching for "beautiful". You may see neighbors like "wonderful". 

Note: your results may be a bit different, depending on how weights were randomly initialized before training the embedding layer.

Note: experimentally, you may be able to produce more interpretable embeddings by using a simpler model. Try deleting the `Dense(16)` layer, retraining the model, and visualizing the embeddings again. -->

Lúc này các embedding đã huấn luyện sẽ được hiển thị lên màn hình. Bạn có thể sử dụng chức năng tìm kiếm từ để xem các từ khác nằm gần nó nhất. Chẳng hạn, nếu thử với từ "beautiful", ta sẽ thấy các từ gần đó chẳng hạn như "wonderful".

Lưu ý: Kết quả mỗi người thu được có thể khác nhau đôi chút tùy thuộc vào các trọng số được khởi tạo ngẫu nhiên trước khi huấn luyện lớp embedding.

Lưu ý: Một số thực nghiệm sử dụng mô hình đơn giản hơn có thể cho kết quả embedding tốt hơn. Hãy thử xóa lớp `Dense (16)`, huấn luyện lại mô hình và trực quan hóa các embedding một lần nữa để kiểm tra.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=1" alt="Screenshot of the embedding projector" width="400"/>

## Tiếp theo

<!-- This tutorial has shown you how to train and visualize word embeddings from scratch on a small dataset.

* To learn about recurrent networks see the [Keras RNN Guide](../../guide/keras/rnn.ipynb).

* To learn more about text classification (including the overall workflow, and if you're curious about when to use embeddings vs one-hot encodings) we recommend this practical text classification [guide](https://developers.google.com/machine-learning/guides/text-classification/step-2-5). -->

Trong hướng dẫn này, chúng ta đã đi qua các bước để huấn luyện và trực quan hóa các word embedding từ đầu trên một tập dữ liệu nhỏ.

* Để tìm hiểu thêm về mạng hồi quy, xem thêm tại [Keras RNN Guide](../../guide/keras/rnn.ipynb).

* Để tìm hiểu thêm về bài toán phân loại văn bản (bao gồm cách thức thực hiện hoặc cho những bạn tò mò khi nào nên sử dụng embedding và khi nào nên sử dụng one-hot encoding), hãy xem qua [Hướng dẫn phân loại văn bản](https://developers.google.com/machine-learning/guides/text-classification/step-2-5).